In [1]:
import random
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

training = []

output_empty = [0] * len(classes)
for doc in documents:
    
    bag = []
    
    pattern_words = doc[0]

    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)

train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

print("Training data created")
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("Model created")



49 documents
24 classes ['الأشعه التشخصيه', 'الأطفال', 'الأنف والأذن', 'التحيه', 'الجراحه العامه', 'الجلدية', 'الشكر', 'العظام', 'العيون', 'القلب', 'المسالك البولية', 'النساء والتوليد', 'جراحة القلب والصدر', 'جراحة المخ والاعصاب', 'جراحة امراض السمنه', 'جراحة عامة أطفال', 'جراحه الثدي ', 'جهاز هضمي اطفال', 'رسم العصب', 'صدر أطفال وامراض الدم', 'قلب أطفال', 'مخ و اعصاب أطفال', 'مواعيد', 'نفسية وعصبية']
79 unique lemmatized words ['؟', 'أشعه', 'أطفال', 'أمراض', 'ازيك', 'اسأل', 'استفساري', 'استفسر', 'اشعه', 'اطفال', 'الأذن', 'الأطفال', 'الأنف', 'الاذن', 'الاشعه', 'الانف', 'البولية', 'التوليد', 'الثدي', 'الجراحه', 'الجلدية', 'الجلديه', 'الدكاتره', 'الدم', 'السلام', 'السمنه', 'العامة', 'العصب', 'العظام', 'العيادات', 'العيون', 'القلب', 'المخ', 'المساعده', 'المسالك', 'النسا', 'النفسيه', 'ايه', 'بخصوص', 'تمام', 'توليد', 'جدا', 'جراحة', 'جراحه', 'جلدية', 'جلديه', 'جهاز', 'حاجه', 'حد', 'دلوقتي', 'رسم', 'سمحت', 'شكرا', 'صدر', 'عامة', 'عايز', 'عايزه', 'عصبية', 'عظام', 'علي', 'عليكم', 'عن', 'قلب',

Epoch 1/200



10/10 [==============================] - 1s 5ms/step - loss: 3.1836 - accuracy: 0.0408
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 3.0928 - accuracy: 0.1020
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 3.0029 - accuracy: 0.1224
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.9472 - accuracy: 0.1429
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.8178 - accuracy: 0.2245
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7898 - accuracy: 0.2041
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 2.7128 - accuracy: 0.1837
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 2.5759 - accuracy: 0.2653
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 2.5326 - accuracy: 0.2653
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 2.4808 - accuracy: 0.3061
Epoch 11/200
10/10 [=

C:\Users\Zaghloula\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
